<a href="https://colab.research.google.com/github/adam1brownell/ai_debate/blob/main/ai_debate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>